# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file used to insert data into the  Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)
    

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)
    
    

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)



### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# Create table session_item_hist for query1
# Partition keys are sessionId and itemInSession

query1 = "CREATE TABLE IF NOT EXISTS session_item_hist"
query1 = query1 + "(userId int, sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
       session.execute(query1)
       print("session_item_hist TABLE has been created successfully")
except Exception as e:
        print(e)
                    

session_item_hist TABLE has been created successfully


In [10]:
# Insert data into session_item_hist table for query1
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1 = "INSERT INTO session_item_hist (userId, sessionId, itemInSession, artist, song, length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Execute the query1
query1 = "select artist, song, length from session_item_hist WHERE sessionId = 338 AND itemInSession = 4"
try:
       rows = session.execute(query1)
except Exception as e:
        print(e)
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
# Create table user_session_hist for query2
# Partition keys are userId and sessionId. Clustering columns is itemInSession.

query2 = "CREATE TABLE IF NOT EXISTS user_session_hist"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
       session.execute(query2)
       print("user_session_hist TABLE has been created successfully")
except Exception as e:
        print(e)

                    

user_session_hist TABLE has been created successfully


In [13]:
# Insert data into user_session_hist table for query2
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO user_session_hist (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

In [14]:
# Execute the query2

query2 = "select artist, song, firstName, lastName from user_session_hist WHERE userId = 10 AND sessionId = 182"
try:
       rows = session.execute(query2)
except Exception as e:
        print(e)
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
# Create table song_title for query3
# Partition keys are song and userId

query3 = "CREATE TABLE IF NOT EXISTS song_title"
query3 = query3 + "(userId int, firstName text, lastName text, song text, PRIMARY KEY (song, userId))"
try:
       session.execute(query3)
       print("song_title TABLE has been created successfully")
except Exception as e:
        print(e)
        

song_title TABLE has been created successfully


In [16]:
# Insert data into user_session_hist table for query3
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO song_title (userId, firstName, lastName, song)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (int(line[10]), str(line[1]), str(line[4]), str(line[9])))
        

In [17]:
# Execute the query3

query3 = "select userId, firstName, lastName from song_title WHERE song = 'All Hands Against His Own'"
try:
       rows = session.execute(query3)
except Exception as e:
        print(e)
for row in rows:
    print (row.firstname, row.lastname)
    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions
query1 = "drop table session_item_hist"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

query2 = "drop table user_session_hist"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

query3 = "drop table song_title"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()